# Message

## 基本用法

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("openai:gpt-5-nano")

system_msg = SystemMessage("你是一个专业的助手")
human_msg = HumanMessage("你好，你是谁？")

# Use with chat models
messages = [system_msg, human_msg]

print(messages)
response = model.invoke(messages)  # Returns AIMessage

print(response)


### Text prompts

In [ ]:
response = model.invoke("Write a haiku about spring")

### Message prompts

In [ ]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage("你是一个专业的诗人"),
    HumanMessage("写一首关于春天的俳句"),
    AIMessage("春风拂面来，花开满园香，燕子归故里。"),
    HumanMessage("再写一首关于夏天的俳句"),
]
response = model.invoke(messages)
print(response)


### Dictionary format 字典格式

In [ ]:
messages = [
    {"role": "system", "content": "You are a poetry expert"},
    {"role": "user", "content": "Write a haiku about spring"},
    {"role": "assistant", "content": "Cherry blossoms bloom..."}
]
response = model.invoke(messages)

## 消息类型

 - 系统消息 - 告诉模型如何表现并为交互提供上下文
 - 用户消息 - 代表用户输入和与模型的交互
 - 助手消息 - 模型生成的响应，包括文本内容、工具调用和元数据
 - 工具消息 - 表示工具调用的输出

### System Messages 系统消息

In [ ]:
# 系统消息
system_msg = SystemMessage("You are a helpful coding assistant.")

messages = [
    system_msg,
    HumanMessage("How do I create a REST API?")
]
response = model.invoke(messages)

In [ ]:
# 系统消息
from langchain.messages import SystemMessage, HumanMessage

system_msg = SystemMessage("""
您是一名高级Python开发人员，具有Web框架方面的专业知识。
始终提供代码示例并解释您的推理。
你的解释要简洁而透彻。
""")

messages = [
    system_msg,
    HumanMessage("如何创建一个REST API？")
]
response = model.invoke(messages)
print(response.content)


### User Messages 用户消息

In [ ]:
response = model.invoke([
  HumanMessage("What is machine learning?")
])

# Message metadata 消息元数据
human_msg = HumanMessage(
    content="Hello!",
    name="alice",  # Optional: identify different users
    id="msg_123",  # Optional: unique identifier for tracing
)

### AI Messages

In [ ]:
response = model.invoke("Explain AI")
print(type(response))  # <class 'langchain_core.messages.AIMessage'>

In [ ]:
from langchain.messages import AIMessage, SystemMessage, HumanMessage

# Create an AI message manually (e.g., for conversation history)
ai_msg = AIMessage("I'd be happy to help you with that question!")

# Add to conversation history
messages = [
    SystemMessage("You are a helpful assistant"),
    HumanMessage("Can you help me?"),
    ai_msg,  # Insert as if it came from the model
    HumanMessage("Great! What's 2+2?")
]

response = model.invoke(messages)
print(response.content)

In [42]:
from langchain.chat_models import init_chat_model
import os
import requests
from dotenv import load_dotenv
from langchain.tools import tool

# 1. 加载环境变量
load_dotenv()

AMAP_KEY = os.getenv("AMAP_KEY")

model = init_chat_model("openai:gpt-5-nano")

# 2. 定义查询天气的工具
@tool
def get_weather(city: str) -> str:
    """
    使用高德地图 Web API 查询中国城市的实时天气。
    参数:
        city: 城市名称（中文，如 "北京"、"深圳"）。
    返回:
        城市的实时天气描述，包括天气现象、温度、风力等。
    """
    url = f"https://restapi.amap.com/v3/weather/weatherInfo?key={AMAP_KEY}&city={city}&extensions=base&output=JSON"
  
    resp = requests.get(url).json()

    if resp.get("status") != "1":
        return f"无法查询到 {city} 的天气信息。错误信息: {resp}"

    lives = resp.get("lives", [])
    if not lives:
        return f"没有找到 {city} 的天气数据。"

    weather_info = lives[0]
    weather = weather_info["weather"]
    temperature = weather_info["temperature"]
    wind = weather_info["winddirection"]
    wind_power = weather_info["windpower"]
    humidity = weather_info["humidity"]
    report_time = weather_info["reporttime"]

    return (
        f"{city} 当前天气：{weather}，气温 {temperature}°C，"
        f"风向 {wind}，风力 {wind_power} 级，湿度 {humidity}%。"
        f"（数据更新时间：{report_time}）"
    )

model_with_tools = model.bind_tools([get_weather])
response = model_with_tools.invoke("What's the weather in Shenzhen?")

# 查看 token 使用情况
print(response.usage_metadata)
print(response.content)

results = []
for tool_call in response.tool_calls:
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")
    print(f"ID: {tool_call['id']}")
    
    if tool_call['name'] == 'get_weather':
        result = get_weather.invoke(tool_call)
        results.append(result)
print("Tool results:", results)

{'input_tokens': 176, 'output_tokens': 216, 'total_tokens': 392, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}}

Tool: get_weather
Args: {'city': '深圳'}
ID: call_p9sWi6HsgKCIlyuRNJQo5iwt
Tool results: [ToolMessage(content='深圳 当前天气：阴，气温 25°C，风向 东北，风力 ≤3 级，湿度 74%。（数据更新时间：2025-10-30 21:00:38）', name='get_weather', tool_call_id='call_p9sWi6HsgKCIlyuRNJQo5iwt')]


In [ ]:
# 流式传输

chunks = []
full_message = None
for chunk in model.stream("Hi"):
    chunks.append(chunk)
    print(chunk.text)
    # print(chunk.text, end="", flush=True)
    full_message = chunk if full_message is None else full_message + chunk

print(full_message)

### Tool Messages 工具消息

In [44]:
# After a model makes a tool call
from langchain.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage

ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "Shenzhen"},
        "id": "call_123"
    }]
)

# Execute tool and create result message
weather_result = "深圳 当前天气：阴，气温 25°C，风向 东北，风力 ≤3 级，湿度 74%。（数据更新时间：2025-10-30 21:00:38）"
tool_message = ToolMessage(
    content=weather_result,
    tool_call_id="call_123"  # Must match the call ID
)

# Continue conversation
messages = [
    HumanMessage("What's the weather in Shenzhen?"),
    ai_message,  # Model's tool call
    tool_message,  # Tool execution result
]
response = model.invoke(messages)  # Model processes the result

response.pretty_print()

================================== Ai Message ==================================

Current weather in Shenzhen:
- Conditions: Cloudy/overcast
- Temperature: 25°C
- Wind: Light breeze from the northeast (≤3 on the Beaufort scale)
- Humidity: 74%
- Data updated: 21:00:38 on 2025-10-30


## Message Content 消息内容

LangChain 聊天模型接受 content 属性中的消息内容，并可以包含：
- 一个字符串
- 以提供者原生格式的内容块列表
- LangChain 的标准内容块列表

In [48]:
from langchain.messages import HumanMessage

# String content
human_message = HumanMessage("Hello, how are you?")

# Provider-native format (e.g., OpenAI)
human_message = HumanMessage(content=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
])

# List of standard content blocks
human_message = HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://example.com/image.jpg"},
])

print(human_message)

content=[{'type': 'text', 'text': 'Hello, how are you?'}, {'type': 'image', 'url': 'https://example.com/image.jpg'}] additional_kwargs={} response_metadata={}


### Standard content blocks 标准内容块

In [51]:
from langchain.messages import AIMessage

message = AIMessage(
    content=[
        {
            "type": "reasoning",
            "id": "rs_abc123",
            "summary": [
                {"type": "summary_text", "text": "summary 1"},
                {"type": "summary_text", "text": "summary 2"},
            ],
        },
        {"type": "text", "text": "...", "id": "msg_abc123"},
    ],
    response_metadata={"model_provider": "openai"}
)
print(message)
print(message.content_blocks) # 标准化的结构化响应；
print(message.content) # 保留原始内容（兼容旧版）


content=[{'type': 'reasoning', 'id': 'rs_abc123', 'summary': [{'type': 'summary_text', 'text': 'summary 1'}, {'type': 'summary_text', 'text': 'summary 2'}]}, {'type': 'text', 'text': '...', 'id': 'msg_abc123'}] additional_kwargs={} response_metadata={'model_provider': 'openai'}
[{'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 1'}, {'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 2'}, {'type': 'text', 'text': '...', 'id': 'msg_abc123'}]


## message.content_blocks 为什么能访问但类里没有显式定义

实际上，在新版 LangChain 中，AIMessage 对象确实没有显式定义 content_blocks 字段，
但 LangChain 在 __getattr__ 中做了动态属性映射。

源码机制大概类似：

```python
def __getattr__(self, name):
    if name == "content_blocks":
        return self.content
    raise AttributeError
```


也就是说：

content_blocks 是一个别名，用于统一兼容不同模型（例如 OpenAI、Anthropic 等）返回的多段内容结构；

内部依然访问 content 字段，只是为了兼容外部调用接口。

### Multimodal 多模态

In [52]:
# From URL
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "url": "https://example.com/path/to/image.jpg"},
    ]
}

# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {
            "type": "image",
            "base64": "AAAAIGZ0eXBtcDQyAAAAAGlzb21tcDQyAAACAGlzb2...",
            "mime_type": "image/jpeg",
        },
    ]
}

# From provider-managed File ID
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "file_id": "file-abc123"},
    ]
}

In [53]:
# From URL
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this document."},
        {"type": "file", "url": "https://example.com/path/to/document.pdf"},
    ]
}

# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this document."},
        {
            "type": "file",
            "base64": "AAAAIGZ0eXBtcDQyAAAAAGlzb21tcDQyAAACAGlzb2...",
            "mime_type": "application/pdf",
        },
    ]
}

# From provider-managed File ID
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this document."},
        {"type": "file", "file_id": "file-abc123"},
    ]
}

In [54]:
# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this audio."},
        {
            "type": "audio",
            "base64": "AAAAIGZ0eXBtcDQyAAAAAGlzb21tcDQyAAACAGlzb2...",
            "mime_type": "audio/wav",
        },
    ]
}

# From provider-managed File ID
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this audio."},
        {"type": "audio", "file_id": "file-abc123"},
    ]
}

In [55]:
# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this video."},
        {
            "type": "video",
            "base64": "AAAAIGZ0eXBtcDQyAAAAAGlzb21tcDQyAAACAGlzb2...",
            "mime_type": "video/mp4",
        },
    ]
}

# From provider-managed File ID
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this video."},
        {"type": "video", "file_id": "file-abc123"},
    ]
}